In [61]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
Decision_Tree = tree.DecisionTreeClassifier()
X = np.load('X_train.npy')
X = X.reshape(60000,784)
y = np.load('y_train.npy')
print(y)
np.save("ab",X)

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42)
Decision_Tree.fit(X_train,y_train)

y_pred = Decision_Tree.predict(X_test)
print("Accuracy :" ,accuracy_score(y_test,y_pred))



[5 0 4 ... 5 6 8]
Accuracy : 0.86975


In [62]:
# ENTROPY CALCULATION
import numpy as np
def Entropy(labels):
    entropy = 0
    size = labels.size
    numbers = {1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,0:0}
    for x in labels:
        numbers[x] += 1
    for key in numbers.keys():
        if (numbers[key]):
            entropy +=(numbers[key]/float(size)) *np.log2(numbers[key]/float(size))
    return entropy


In [63]:
Entropy(np.asarray([1,2,3,4,0,1]))

-2.2516291673878226

In [64]:
def compute_entropy_threshold(data , index,labels):
    entropy_labels = Entropy(labels)
    computed_values = []
    feature = data[:,index]
    max_info = 0
    for I in range(0,feature.size):
        arr1 = np.empty(0)
        arr2 = np.empty(0)
        entropy_drop = 0
        
        if (feature[I] not in computed_values):
            computed_values.append(feature[I])
            for j in range(0,labels.size):
                if (feature[j] < feature[I]):
                    arr1 = np.append(arr1,labels[j])
                else:
                    arr2 = np.append(arr2,labels[j])
            if (arr1.size > 0 and arr2.size>0):
                entropy_drop = entropy_labels - ((arr1.size)/labels.size)*Entropy(arr1) - ((arr1.size)/labels.size)*Entropy(arr2)
            if (entropy_drop > max_info):
                threshold = feature[I]
                split_arr1 = arr1
                split_arr2 = arr2
                max_info = entropy_drop
    if (max_info > 0):
        return max_info,threshold,split_arr1,split_arr2
    else:
        return 0,0,0,0   


In [65]:
def split_data(index,threshold,data):   
    return data[(data[:,index] >= threshold)],data[(data[:,index])< threshold]

In [66]:
def best_split(data,labels):
    max_info = 0
    pos = 0
    threshold = 0
    for index in range(784):
        info,cur_threshold,split1,split2 =compute_entropy_threshold(data,index,labels)
        if (info > max_info):
            pos = index
            threshold = cur_threshold
            label1 = split1
            label2 = split2
            max_info = info
    return pos,threshold
    

In [67]:
class DecisionTreeNode:
    """Class for a decision tree node."""
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

In [68]:
class DecisionTreeNode:
    """Class for a decision tree node."""
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTree:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.root = None

    def fit(self, data, labels, depth=0):
        """Recursively builds the decision tree."""
        # Base case: Stop splitting if pure or max depth reached
        if depth == self.max_depth or len(np.unique(labels)) == 1: #Why?
            return DecisionTreeNode(value=np.bincount(labels).argmax()) #Why?

        # Find best feature and threshold
        best_feature, best_threshold = best_split(data, labels)

        # Partition data
        left_mask = data[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        if np.all(left_mask) or np.all(right_mask):  # Prevent infinite splitting
            return DecisionTreeNode(value=np.bincount(labels).argmax())

        left_child = self.fit(data[left_mask], labels[left_mask], depth + 1)
        right_child = self.fit(data[right_mask], labels[right_mask], depth + 1)

        return DecisionTreeNode(feature=best_feature, threshold=best_threshold, left=left_child, right=right_child)

    def train(self, data, labels):
        """Initialize tree training."""
        self.root = self.fit(data, labels, depth=0)

In [69]:
def predict_sample(node, sample):
    """Recursively predict the label for a single sample."""
    if node.value is not None:
        return node.value
    if sample[node.feature] <= node.threshold:
        return predict_sample(node.left, sample)
    else:
        return predict_sample(node.right, sample)

def predict(tree, data):
    """Predict labels for multiple samples."""
    return np.array([predict_sample(tree.root, sample) for sample in data])


In [76]:
tree = DecisionTree()
X = np.load('X_train.npy')
y = np.load('y_train.npy')
X = X.reshape(60000,784)
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.01,test_size=0.01)
tree.train(X_train,y_train)
accuracy_score(predict(tree,X_test),y_test)

0.12833333333333333

In [83]:
X = np.load('X_train.npy')
y = np.load('y_train.npy')
X = X.reshape(60000,784)
X[:,0]*1 == X[:,0]

FileNotFoundError: [Errno 2] No such file or directory: 'X_train.npy'